In [ ]:
# data loading
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# for better display of plots
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
#pd.set_option('display.max_rows', None) 

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("../data/final_cohort_main_df.csv")

In [ ]:
df.head()

In [ ]:
print(df.shape)
df_dupes = df[df.duplicated(subset=['SUBJECT_ID'], keep=False)].sort_values(by='SUBJECT_ID')
print(df_dupes.shape)
print(df_dupes.head(20))

In [ ]:
df.shape

In [ ]:
# to reduce the name of the columns -> we remove the sufix _val
df = df.rename(
    columns=lambda c: c[:-4] if c.endswith('_val') else c
)

if 'GENDER' in df.columns:
    df['is_male'] = df['GENDER'].map({'M': 1, 'F': 0})
    
df.drop(columns = ['GENDER'], inplace = True)

In [ ]:
print(f"There is {df['is_nonsurvivor'].mean()*100:.2f}% of admissions with the patient not surviving.")

# Reducing the number of columns

vasopressor, mechanical_ventilation and rrt_dialysis contain no missing values because they were previously coalesced to 0.

## % of nans for each column

In [ ]:
missing_df = pd.DataFrame({
    'variable': df.columns,
    'n_missing': [df[col].isna().sum() for col in df.columns],
    'pct_missing': [df[col].isna().mean()*100 for col in df.columns]
})

# order by porcentage
missing_df = missing_df.sort_values(by='pct_missing', ascending=False)

print(missing_df.to_string())
missing_df.to_csv("pct_missing.csv", index = False)

In [ ]:
missing_threshold = 85

missing_pct = df.isna().mean() * 100
cols_to_drop = missing_pct[missing_pct >= missing_threshold].index
print(f"{len(cols_to_drop)} columns would be dropped due to high missing values (>{missing_threshold}%):\n")
print(list(cols_to_drop))

### Unification of similar or equal variables

Some labels from chartevents or labevents or vital signs were codified with different names resulting in different columns for each label name.

#### Temperature

In [ ]:
# fahrenheit -> celsius
def convert_f_to_c(f_val):
    return (f_val - 32) * 5.0/9.0

cols_c = [c for c in df.columns if 'Temperature' in c and ('C_' in c or 'Celsius' in c) and 'max' in c]
cols_f = [c for c in df.columns if 'Temperature' in c and ('F_' in c or 'Fahrenheit' in c) and 'max' in c]

df.drop(columns=cols_c + cols_f, inplace=True)

# we do the same for the minimums and maximums
for suf in ['mean', 'min']:
    df['Vital_Temp_Unified_' + suf] = np.nan
    cols_c = [c for c in df.columns if 'Temperature' in c and ('C_' in c or 'Celsius' in c) and suf in c]
    cols_f = [c for c in df.columns if 'Temperature' in c and ('F_' in c or 'Fahrenheit' in c) and suf in c]
    for c in cols_c:
        df['Vital_Temp_Unified_' + suf] = df['Vital_Temp_Unified_' + suf].fillna(df[c])
    for c in cols_f:
        df['Vital_Temp_Unified_' + suf] = df['Vital_Temp_Unified_' + suf].fillna(df[c].apply(convert_f_to_c))
    
    df.drop(columns=cols_c + cols_f, inplace=True)

missing_pct = df['Vital_Temp_Unified_mean'].isna().mean() * 100
print(f"Vital_Temp_Unified has {missing_pct:.2f}% of missing values")


#### Respiratory Rate 

In [ ]:
print(df[['Vital_Respiratory Rate (Total)_mean', 'Vital_Resp Rate (Total)_mean', 'Vital_Respiratory Rate_mean']].head(10))

for suf in ['_mean', '_min', '_max']:
    df['Vital_Respiratory Rate (Total) Unified' + suf] = df['Vital_Respiratory Rate (Total)' + suf].combine_first(
        df['Vital_Resp Rate (Total)' + suf]
    )  
    df.drop(columns=['Vital_Resp Rate (Total)'+ suf, 'Vital_Respiratory Rate (Total)' + suf], inplace=True)
    
missing_pct = df['Vital_Respiratory Rate (Total) Unified_mean'].isna().mean() * 100
print(f"Vital_Respiratory Rate (Total) Unified has {missing_pct:.2f}% of missing values")


#### Blood Pressure

In [ ]:
bp_cols = ['Vital_Manual BP [Systolic]_max', 'Vital_Manual BP [Diastolic]_max', 'Vital_Arterial BP #2 [Systolic]_max',
           'Vital_Arterial BP #2 [Diastolic]_max', 'Vital_Arterial BP Mean #2_max','Vital_Arterial Blood Pressure mean_mean',
           'Vital_Arterial BP Mean_max', 'Vital_Arterial BP [Systolic]_max', 'Vital_Arterial BP [Diastolic]_max',
           'Vital_ART BP mean_max', 'Vital_Arterial Blood Pressure systolic_max', 'Vital_Arterial Blood Pressure diastolic_max']
print(df[bp_cols].head())

for suf in ['_mean', '_min', '_max']:
    df['Vital_Mean Arterial Pressure Unified' + suf] = np.nan

    # arterial BP mean first (There are 3 types of codified label)
    df['Vital_Mean Arterial Pressure Unified' + suf] = df['Vital_Mean Arterial Pressure Unified' + suf].fillna(df['Vital_Arterial Blood Pressure mean' + suf])
    df['Vital_Mean Arterial Pressure Unified' + suf] = df['Vital_Mean Arterial Pressure Unified' + suf].fillna(df['Vital_Arterial BP Mean' + suf])
    df['Vital_Mean Arterial Pressure Unified' + suf] = df['Vital_Mean Arterial Pressure Unified' + suf].fillna(df['Vital_ART BP mean' + suf])
    
    # arterial BP mean #2
    df['Vital_Mean Arterial Pressure Unified' + suf] = df['Vital_Mean Arterial Pressure Unified' + suf].fillna(df['Vital_Arterial BP Mean #2' + suf])
    
    # arterial BP mean computed from arterial BP systolic and diastolic (there are 2 codified labels)
    df['Vital_Mean Arterial Pressure Unified' + suf] = df['Vital_Mean Arterial Pressure Unified' + suf].fillna(
            (df['Vital_Arterial Blood Pressure systolic' + suf] + 2 * df['Vital_Arterial Blood Pressure diastolic' + suf]) / 3
        )
    df['Vital_Mean Arterial Pressure Unified' + suf] = df['Vital_Mean Arterial Pressure Unified' + suf].fillna(
            (df['Vital_Arterial BP [Systolic]' + suf] + 2 * df['Vital_Arterial BP [Diastolic]' + suf]) / 3
        )
    
    # arterial mean computed from arterial BP systolic and diastolic #2
    df['Vital_Mean Arterial Pressure Unified' + suf] = df['Vital_Mean Arterial Pressure Unified' + suf].fillna(
            (df['Vital_Arterial BP #2 [Systolic]' + suf] + 2 * df['Vital_Arterial BP #2 [Diastolic]' + suf]) / 3
        )
    
    # arterial mean computed from manual BP (computed)
    df['Vital_Mean Arterial Pressure Unified' + suf] = df['Vital_Mean Arterial Pressure Unified' + suf].fillna(df['Vital_Manual BP Mean(calc)' + suf])

    # arterial mean computed from manual BP systolic and diastolic 
    df['Vital_Mean Arterial Pressure Unified' + suf] = df['Vital_Mean Arterial Pressure Unified' + suf].fillna(
            (df['Vital_Manual BP [Systolic]' + suf] + 2 * df['Vital_Manual BP [Diastolic]' + suf]) / 3
        )

bp_cols_to_drop = [
    'Vital_Manual BP [Systolic]',
    'Vital_Manual BP [Diastolic]',
    'Vital_Arterial BP #2 [Systolic]',
    'Vital_Arterial BP #2 [Diastolic]',
    'Vital_Arterial BP Mean #2',
    'Vital_ART BP mean',
    'Vital_Arterial BP Mean', 
    'Vital_Arterial BP [Systolic]', 
    'Vital_Arterial BP [Diastolic]',
    'Vital_Arterial Blood Pressure mean',
    'Vital_Arterial Blood Pressure systolic',
    'Vital_Arterial Blood Pressure diastolic',
    'Vital_Manual BP Mean(calc)'
]
cols_to_drop = [c + suf for c in bp_cols_to_drop for suf in ['_mean', '_min', '_max'] if c + suf in df.columns]
df.drop(columns=cols_to_drop, inplace=True)


missing_pct = df['Vital_Mean Arterial Pressure Unified_mean'].isna().mean() * 100
print(f"Vital_Mean Arterial Pressure Unified has {missing_pct:.2f}% of missing values")

### Sodium, Creatinine, Chloride, Potassium, Blood Urean NItrogen(BUN), Bilirubin,...

In [ ]:
vars_electrolytes = ['Sodium', 'Creatinine', 'Chloride', 'Potassium', 'Magnesium', 'Bilirubin', 'Glucose']

for var in vars_electrolytes:
    cols = [c for c in df.columns if var in c and 'max' in c]
    print(df[cols].head())

cols = [c for c in df.columns if ('Urea Nitrogen' in c or 'BUN' in c) and 'max' in c]
print(df[cols].head())

cols = [c for c in df.columns if ('RBC' in c or 'Red Blood Cells' in c) and 'max' in c]
print(df[cols].head())

cols = [c for c in df.columns if 'Platelet' in c and 'max' in c]
print(df[cols].head())

In [ ]:
# 1. Electrolytes and lab variables to unify
vars_electrolytes = ['Sodium', 'Creatinine', 'Chloride', 'Potassium', 'Magnesium', 'Bilirubin', 'Glucose']

for var in vars_electrolytes:
    for suf in ['max', 'mean', 'min']:
        # Find all columns containing the variable and suffix
        cols = [c for c in df.columns if var in c and suf in c]
        if not cols:
            continue

        # Prioritize columns starting with 'Lab_'
        lab_cols = [c for c in cols if c.startswith('Lab_')]
        other_cols = [c for c in cols if not c.startswith('Lab_')]
        ordered_cols = lab_cols + other_cols  # Lab first

        print(f"Unifying {var}_{suf}: {cols}")
        
        # Create a unified column
        unified_col = f'Lab_{var}_Unified_{suf}'
        df[unified_col] = np.nan
        
        # Fill NaNs with values from each column
        for col in ordered_cols:
            df[unified_col] = df[unified_col].fillna(df[col])

        df.drop(columns=cols, inplace=True)

# 2. Urea Nitrogen / BUN
for suf in ['max', 'mean', 'min']:
    cols = [c for c in df.columns if ('Urea Nitrogen' in c or 'BUN' in c) and suf in c]
    if not cols:
        continue

    # Prioritize columns starting with 'Lab_'
    lab_cols = [c for c in cols if c.startswith('Lab_')]
    other_cols = [c for c in cols if not c.startswith('Lab_')]
    ordered_cols = lab_cols + other_cols  # Lab first

    print(f"Unifying Urea/BUN_{suf}: {cols}")
    
    unified_col = f'Lab_Urea_Unified_{suf}'
    df[unified_col] = np.nan
    for col in ordered_cols:
        df[unified_col] = df[unified_col].fillna(df[col])

    df.drop(columns=cols, inplace=True)

# 3. Red Blood Cells (RBC)
for suf in ['max', 'mean', 'min']:
    cols = [c for c in df.columns if ('RBC' in c or 'Red Blood Cells' in c) and suf in c]
    if not cols:
        continue

    # Prioritize columns starting with 'Lab_'
    lab_cols = [c for c in cols if c.startswith('Lab_')]
    other_cols = [c for c in cols if not c.startswith('Lab_')]
    ordered_cols = lab_cols + other_cols  # Lab first

    print(f"Unifying RBC_{suf}: {cols}")
    
    unified_col = f'Lab_Red_Blood_Cells_Unified_{suf}'
    df[unified_col] = np.nan
    for col in ordered_cols:
        df[unified_col] = df[unified_col].fillna(df[col])

    df.drop(columns=cols, inplace=True)

# 4. Platelets
for suf in ['max', 'mean', 'min']:
    cols = [c for c in df.columns if 'Platelet' in c and suf in c]
    if not cols:
        continue
    
    # Prioritize columns starting with 'Lab_'
    lab_cols = [c for c in cols if c.startswith('Lab_')]
    other_cols = [c for c in cols if not c.startswith('Lab_')]
    ordered_cols = lab_cols + other_cols  # Lab first

    print(f"Unifying Platelets_{suf}: {cols}")
    
    unified_col = f'Lab_Platelets_Unified_{suf}'
    df[unified_col] = np.nan
    for col in ordered_cols:
        df[unified_col] = df[unified_col].fillna(df[col])

    df.drop(columns=cols, inplace=True)

print("Original columns dropped, only unified columns remain.")

In [ ]:
df.shape

In [ ]:
missing_threshold = 80

missing_pct = df.isna().mean() * 100
cols_to_drop = missing_pct[missing_pct > missing_threshold].index
print(f"{len(cols_to_drop)} columns dropped due to high missing values (>{missing_threshold}%):\n")
print(list(cols_to_drop))

In [ ]:
# summary = df.describe().T
# summary['cv'] = summary['std'] / summary['mean'] # Coefficient of Variation
# summary

In [ ]:
df['Vital_Mental status_mean'].value_counts()

## Removing columns of min and max

To reduce dimensionality and collinearity, minimum and maximum values were removed for variables with minimal intra-patient variability during the first 24 hours, retaining only the mean value.

In [ ]:
# import re

# threshold = 0.99        # relative similarity threshold
# min_fraction = 0.10     # minimum fraction of valid rows required
# agreement = 0.90       # required agreement percentage

# cols = pd.Index(df.columns)

# # Identify base variable names (without _mean/_min/_max)
# base_vars = set(
#     re.sub(r'_(mean|min|max)$', '', c)
#     for c in cols
#     if re.search(r'_(mean|min|max)$', c)
# )

# cols_to_drop = []

# for var in base_vars:
#     mean_col = f"{var}_mean"
#     min_col  = f"{var}_min"
#     max_col  = f"{var}_max"

#     # Skip if not all components exist
#     if not all(c in cols for c in [mean_col, min_col, max_col]):
#         continue

#     mean_vals = df[mean_col]
#     min_vals  = df[min_col]
#     max_vals  = df[max_col]

#     # mean != 0 → use relative ratio
#     valid_ratio = (
#         mean_vals.notna() &
#         min_vals.notna() &
#         max_vals.notna() &
#         (mean_vals != 0)
#     )

#     drop_case_A = False
#     if valid_ratio.mean() >= min_fraction:
#         min_ratio = (min_vals[valid_ratio] / mean_vals[valid_ratio]).abs()
#         max_ratio = (max_vals[valid_ratio] / mean_vals[valid_ratio]).abs()

#         drop_case_A = (
#             (min_ratio > threshold).mean() > agreement and
#             (max_ratio > threshold).mean() > agreement
#         )

#     # if mean == 0 → check flat signals
#     valid_zero = (
#         mean_vals.eq(0) &
#         min_vals.notna() &
#         max_vals.notna()
#     )

#     drop_case_B = False
#     if valid_zero.mean() >= min_fraction:
#         drop_case_B = (
#             (min_vals[valid_zero] == 0).mean() > agreement and
#             (max_vals[valid_zero] == 0).mean() > agreement
#         )

#     if drop_case_A or drop_case_B:
#         cols_to_drop.extend([min_col, max_col])

# # Drop redundant columns
# #df_reduced = df.drop(columns=cols_to_drop)

# print(f"Dropped {len(cols_to_drop)} redundant min/max columns: {cols_to_drop}")

## Feature correlation

In [ ]:
cols = [c for c in df.columns if c not in ('SUBJECT_ID', 'HADM_ID', 'ADMITTIME','DISCHTIME', 'is_nonsurvivor')]
print(cols)
corr = df[cols].corr()

In [ ]:
from scipy.stats import ttest_ind

cols = [c for c in df.columns if c not in ('SUBJECT_ID', 'HADM_ID', 'ADMITTIME','DISCHTIME', 'is_nonsurvivor')]
print(cols)

survivors = df[df['is_nonsurvivor'] == 0]
nonsurvivors = df[df['is_nonsurvivor'] == 1]

stats = df.groupby('is_nonsurvivor')[cols].mean().transpose()
stats.columns = ['Survivors (Mean)', 'Non-Survivors (Mean)']

stats['Total Cohort (Mean)'] = df[cols].mean()
stats['% Nans'] = df[cols].isna().mean() * 100

# significance of the differences
pvals = []
for col in cols:
    if col != 'is_nonsurvivor': 
        # remove nans from each group
        x = survivors[col].dropna()
        y = nonsurvivors[col].dropna()
        # test t
        if len(x) > 1 and len(y) > 1:
            p = ttest_ind(x, y, equal_var=False).pvalue
        else:
            p = np.nan
        pvals.append(p)

stats['p-value'] = pvals

stats['Significant'] = stats['p-value'] < 0.05  # True if p<0.05
stats['p-value'] = stats['p-value'].apply(lambda x: f"{x:.2e}" if pd.notna(x) else np.nan)

stats = stats[['Total Cohort (Mean)', 'Survivors (Mean)', 'Non-Survivors (Mean)',
                 '% Nans', 'p-value', 'Significant']]

display(stats)
stats.to_csv('statistics_survivors.csv')

In [ ]:
significant_cols = stats[(stats['Significant']) & (stats['% Nans'] < 60)].index.tolist()
print(len(significant_cols))

In [ ]:
# corr_matrix = df[significant_cols].corr().abs()  # matriz de correlación absoluta
# to_drop = set()
# threshold = 0.99

# for i in range(len(corr_matrix.columns)):
#     for j in range(i):
#         if corr_matrix.iloc[i, j] > threshold:
#             colname = corr_matrix.columns[i]
#             to_drop.add(colname)

# print(f"there are {len(to_drop)} : {to_drop}")
# final_cols = [c for c in significant_cols if c not in to_drop]
# print(f"{len(final_cols)} signficant columns ")

In [ ]:
stats[(stats['% Nans'] < 60) & (stats.index.str.endswith('_mean'))]['Significant'].value_counts()

In [ ]:
significant_mean_cols = stats[(stats['Significant']) & (stats['% Nans'] < 60) & (stats.index.str.endswith('_mean'))].index.tolist()
print(len(significant_mean_cols))

corr_matrix = df[significant_mean_cols].corr().abs() 

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix,
            annot=False,
            fmt=".1f", # Round to 2 decimal places
            cmap='coolwarm',
            center=0,
            vmin=-1,
            vmax=1,
            linewidths=0.5)
plt.title('Feature correlation heatmap')
plt.show()

## Outliers

In [ ]:
q1 = df.quantile(0.25)
q3 = df.quantile(0.75)
iqr = q3 - q1
outlier_counts = ((df < (q1 - 1.5 * iqr)) | (df > (q3 + 1.5 * iqr))).sum()
print("\nNumber of Outliers per Feature:\n", outlier_counts)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Variables significativas
significant_mean_cols = stats[(stats['Significant']) & (stats['% Nans'] < 60) & (stats.index.str.endswith('_mean'))].index.tolist()

# Dividir en varias figuras si son muchas
plots_per_fig = 12
for i in range(0, len(significant_mean_cols), plots_per_fig):
    cols_subset = significant_mean_cols[i:i+plots_per_fig]
    n_cols = 4
    n_rows = int(np.ceil(len(cols_subset) / n_cols))
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(4*n_cols, 3*n_rows))
    axes = axes.flatten()
    
    for ax, col in zip(axes, cols_subset):
        # Use seaborn.histplot
        sns.histplot(
            data=df,
            x=col,
            hue='is_nonsurvivor',
            kde=True,
            palette={0:'lightblue', 1:'plum'}, # survivor=0, non-survivor=1
            alpha=0.6,
            ax=ax,
            element='step',  # step or bars
            common_norm=False # separate normalization for each hue
        )
        ax.set_title(col, fontsize=10)

        # Calculate % of NaNs per group
        pct_nan_surv = df[df['is_nonsurvivor']==0][col].isna().mean() * 100
        pct_nan_nonsurv = df[df['is_nonsurvivor']==1][col].isna().mean() * 100
        
        # Annotate on the plot
        ax.text(
            0.35, 0.95,  # x=0.65 para dejar espacio para la leyenda
            f"NaN %\nSurv: {pct_nan_surv:.1f}%\nNon-Surv: {pct_nan_nonsurv:.1f}%",
            horizontalalignment='left', 
            verticalalignment='top',
            transform=ax.transAxes, 
            fontsize=6, 
            bbox=dict(facecolor='white', alpha=0.5)
        )
    
    # Remove empty axes
    for ax in axes[len(cols_subset):]:
        ax.remove()


    plt.tight_layout()
    plt.show()


In [ ]:
# Significant features
significant_cols = stats[(stats['Significant']) & (stats['% Nans'] < 60)].index.tolist()
print(len(significant_cols))

corr_with_target = df[significant_cols + ['is_nonsurvivor']].corr()['is_nonsurvivor'].drop('is_nonsurvivor').sort_values(ascending=False)
print(corr_with_target.shape)

plt.figure(figsize=(6, 10))
sns.heatmap(
    corr_with_target.to_frame(),
    cmap=sns.diverging_palette(250, 345, as_cmap=True),
    center=0,
    annot=False,
    fmt=".3f"
)
plt.title('Correlation with diagnosis')
plt.show()

### Graphics for the binary features


In [ ]:
### Graphics for the binary features
binary_cols = ['vasopressor', 'mechanical_ventilation', 'rrt_dialysis']


# Imputation of nans

In [ ]:
df_90 = df[df.isna().mean(axis=1) < 0.1]  # <10% missing
df_90.shape 

# w ecan not use the combine method for imputing nans

In [ ]:
# Define groups
groups = {'survivor': 0, 'non_survivor': 1}

# Threshold for deciding median vs mean
outlier_threshold = 50

# Loop over all columns you want to impute
for col in df.columns:
    
    # Skip target and IDs
    if col in ['SUBJECT_ID', 'HADM_ID', 'DISCHTIME', 'ADMITTIME', 'is_nonsurvivor', 'length_of_stay_days', 'vasopressor', 'mechanical_ventilation',	'rrt_dialysis']:
        continue
    
    # Calculate number of extreme outliers (e.g., outside 1.5*IQR)
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers_count = ((df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)).sum()
    
    # Decide imputation function
    if outliers_count > outlier_threshold:
        impute_func = np.median
    else:
        impute_func = np.mean
    
    # Impute per group
    for label, val in groups.items():
        mask = df['is_nonsurvivor'] == val
        impute_value = impute_func(df.loc[mask, col])
        df.loc[mask & df[col].isna(), col] = impute_value


In [ ]:
for col in significant_cols:
    sns.boxplot(
        data=df,
        x='is_nonsurvivor',
        y=col
    )
    plt.show()